<a href="https://colab.research.google.com/github/Minwoo-study/Korean_Classic_DB/blob/main/%ED%95%9C%EA%B5%AD%EA%B3%A0%EC%A0%84%EC%A2%85%ED%95%A9DB_%EC%9B%90%EB%AC%B8_%EB%8B%A8%EC%96%B4_%EB%8B%A8%EC%9C%84_tokenizing%2Bword2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 한국고전종합DB 원문 번역문 word2vec

In [2]:
!pip3 install pickle5
!pip install --upgrade pandas
!pip install -U kiwipiepy gensim kss -q

In [3]:
import pandas as pd
# import pickle5 as pickle
import pickle
import re
import regex
from tqdm import tqdm
tqdm.pandas()

from kiwipiepy import Kiwi, Option
kiwi = Kiwi()
kiwi.prepare()

from kss import split_sentences
import itertools
from collections import defaultdict
from collections import Counter

import matplotlib.pyplot as plt
import matplotlib as mpl
# 그래프에서 마이너스 폰트 깨지는 문제에 대한 대처
mpl.rcParams['axes.unicode_minus'] = False
plt.rc('font', family='NanumBarunGothic')
font_path = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
import matplotlib.colors as clr

from gensim.models import Word2Vec
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

### 원문 Word2vec

In [4]:
#분석용 피클파일 불러오기
# https://drive.google.com/file/d/1BCAZmBi3OK9GFtC2prab4wCRQtJaYUVq/view?usp=sharing
!curl -c ./cookie -s -L "https://drive.google.com/uc?export=download&id=1BCAZmBi3OK9GFtC2prab4wCRQtJaYUVq" > /dev/null
!curl -Lb ./cookie "https://drive.google.com/uc?export=download&confirm=`awk '/download/ {print $NF}' ./cookie`&id=1BCAZmBi3OK9GFtC2prab4wCRQtJaYUVq" -o df_sent.pkl

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   408    0   408    0     0   1533      0 --:--:-- --:--:-- --:--:--  1533
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  674M  100  674M    0     0   117M      0  0:00:05  0:00:05 --:--:--  142M


In [5]:
with open('df_sent.pkl', "rb") as fh:
  df_sent = pickle.load(fh)

In [6]:
df_sent

,ID,원문,번역문,sentence,token
0,ITKC_MO_0001A_0010_010_0010,記有之。曰酒醴之美。而玄酒明水之尙。貴五味之本也。黼黻文繡之美。而疏布之尙。反女功之始也。古...,\n《예기》에 이르기를 “단술이 맛이 좋긴 하지만 제사 때에 현주와 명수 같은 물을...,記有之,"[記, 有, 之]"
0,ITKC_MO_0001A_0010_010_0010,記有之。曰酒醴之美。而玄酒明水之尙。貴五味之本也。黼黻文繡之美。而疏布之尙。反女功之始也。古...,\n《예기》에 이르기를 “단술이 맛이 좋긴 하지만 제사 때에 현주와 명수 같은 물을...,曰酒醴之美,"[曰, 酒, 醴, 之, 美]"
0,ITKC_MO_0001A_0010_010_0010,記有之。曰酒醴之美。而玄酒明水之尙。貴五味之本也。黼黻文繡之美。而疏布之尙。反女功之始也。古...,\n《예기》에 이르기를 “단술이 맛이 좋긴 하지만 제사 때에 현주와 명수 같은 물을...,而玄酒明水之尙,"[而, 玄, 酒, 明, 水, 之, 尙]"
0,ITKC_MO_0001A_0010_010_0010,記有之。曰酒醴之美。而玄酒明水之尙。貴五味之本也。黼黻文繡之美。而疏布之尙。反女功之始也。古...,\n《예기》에 이르기를 “단술이 맛이 좋긴 하지만 제사 때에 현주와 명수 같은 물을...,貴五味之本也,"[貴, 五, 味, 之, 本, 也]"
0,ITKC_MO_0001A_0010_010_0010,記有之。曰酒醴之美。而玄酒明水之尙。貴五味之本也。黼黻文繡之美。而疏布之尙。反女功之始也。古...,\n《예기》에 이르기를 “단술이 맛이 좋긴 하지만 제사 때에 현주와 명수 같은 물을...,黼黻文繡之美,"[黼, 黻, 文, 繡, 之, 美]"
...,...,...,...,...,...
105623,ITKC_MO_1252A_0290_000_0030,嗚呼。此吾響山公遺文也。公大節旣昭然如日星之在天。其不朽何待乎文哉。獨後死者之論。以爲公歿而...,"\n오호라, 이 책은 우리 향산공(響山公)의 유문(遺文)이다. 공의 큰 절의가 이미...",人心世道日趍於汙,"[人, 心, 世, 道, 日, 趍, 於, 汙]"
105623,ITKC_MO_1252A_0290_000_0030,嗚呼。此吾響山公遺文也。公大節旣昭然如日星之在天。其不朽何待乎文哉。獨後死者之論。以爲公歿而...,"\n오호라, 이 책은 우리 향산공(響山公)의 유문(遺文)이다. 공의 큰 절의가 이미...",不知問學之爲何名而義理之爲何物,"[不, 知, 問, 學, 之, 爲, 何, 名, 而, 義, 理, 之, 爲, 何, 物]"
105623,ITKC_MO_1252A_0290_000_0030,嗚呼。此吾響山公遺文也。公大節旣昭然如日星之在天。其不朽何待乎文哉。獨後死者之論。以爲公歿而...,"\n오호라, 이 책은 우리 향산공(響山公)의 유문(遺文)이다. 공의 큰 절의가 이미...",如先生者更於何叫起來也,"[如, 先, 生, 者, 更, 於, 何, 叫, 起, 來, 也]"
105623,ITKC_MO_1252A_0290_000_0030,嗚呼。此吾響山公遺文也。公大節旣昭然如日星之在天。其不朽何待乎文哉。獨後死者之論。以爲公歿而...,"\n오호라, 이 책은 우리 향산공(響山公)의 유문(遺文)이다. 공의 큰 절의가 이미...",只得拊遺卷爲之太息而已,"[只, 得, 拊, 遺, 卷, 爲, 之, 太, 息, 而, 已]"


In [7]:
df_sent = df_sent.reset_index(drop= True)

### 원문에서 같이 보고 싶은 단어 정리

#### 단어가 하나일 때

In [123]:
# # words(token 열의 각 값(리스트형식), 합치고 싶은 단어)

# def words (lists, word):
#   global result
#   result =[]
#   char1 , char2 = list(word)
#   for index, value in enumerate(lists):
#     try: 
#       if (value == char1) & (lists[index+1]==char2):
        
#         result.append(word)
#         print(word)
#       elif (value ==char2)&(lists[index-1]==char1) :
#         continue
#       else :
#         result.append(value)
#     except IndexError :
#       result.append(value)
#       continue
  


In [8]:
# words(token 열의 각 값(리스트형식), 합치고 싶은 단어)
def words (lists, word):
  global result
  char1 , char2 = list(word)
  for index, value in enumerate(lists):
    try: 
      if (value == char1) & (lists[index+1]==char2):
        lists[index] = word
        #print(word)
      elif (value ==char2)&(lists[index-1]==word) :
        del lists[index]
      else :
        continue
    except IndexError :
      result =lists

In [177]:
df_sent

,ID,원문,번역문,sentence,token
0,ITKC_MO_0001A_0010_010_0010,記有之。曰酒醴之美。而玄酒明水之尙。貴五味之本也。黼黻文繡之美。而疏布之尙。反女功之始也。古...,\n《예기》에 이르기를 “단술이 맛이 좋긴 하지만 제사 때에 현주와 명수 같은 물을...,記有之,"[記, 有, 之]"
1,ITKC_MO_0001A_0010_010_0010,記有之。曰酒醴之美。而玄酒明水之尙。貴五味之本也。黼黻文繡之美。而疏布之尙。反女功之始也。古...,\n《예기》에 이르기를 “단술이 맛이 좋긴 하지만 제사 때에 현주와 명수 같은 물을...,曰酒醴之美,"[曰, 酒, 醴, 之, 美]"
2,ITKC_MO_0001A_0010_010_0010,記有之。曰酒醴之美。而玄酒明水之尙。貴五味之本也。黼黻文繡之美。而疏布之尙。反女功之始也。古...,\n《예기》에 이르기를 “단술이 맛이 좋긴 하지만 제사 때에 현주와 명수 같은 물을...,而玄酒明水之尙,"[而, 玄, 酒, 明, 水, 之, 尙]"
3,ITKC_MO_0001A_0010_010_0010,記有之。曰酒醴之美。而玄酒明水之尙。貴五味之本也。黼黻文繡之美。而疏布之尙。反女功之始也。古...,\n《예기》에 이르기를 “단술이 맛이 좋긴 하지만 제사 때에 현주와 명수 같은 물을...,貴五味之本也,"[貴, 五, 味, 之, 本, 也]"
4,ITKC_MO_0001A_0010_010_0010,記有之。曰酒醴之美。而玄酒明水之尙。貴五味之本也。黼黻文繡之美。而疏布之尙。反女功之始也。古...,\n《예기》에 이르기를 “단술이 맛이 좋긴 하지만 제사 때에 현주와 명수 같은 물을...,黼黻文繡之美,"[黼, 黻, 文, 繡, 之, 美]"
...,...,...,...,...,...
3796187,ITKC_MO_1252A_0290_000_0030,嗚呼。此吾響山公遺文也。公大節旣昭然如日星之在天。其不朽何待乎文哉。獨後死者之論。以爲公歿而...,"\n오호라, 이 책은 우리 향산공(響山公)의 유문(遺文)이다. 공의 큰 절의가 이미...",人心世道日趍於汙,"[人, 心, 世, 道, 日, 趍, 於, 汙]"
3796188,ITKC_MO_1252A_0290_000_0030,嗚呼。此吾響山公遺文也。公大節旣昭然如日星之在天。其不朽何待乎文哉。獨後死者之論。以爲公歿而...,"\n오호라, 이 책은 우리 향산공(響山公)의 유문(遺文)이다. 공의 큰 절의가 이미...",不知問學之爲何名而義理之爲何物,"[不, 知, 問, 學, 之, 爲, 何, 名, 而, 義, 理, 之, 爲, 何, 物]"
3796189,ITKC_MO_1252A_0290_000_0030,嗚呼。此吾響山公遺文也。公大節旣昭然如日星之在天。其不朽何待乎文哉。獨後死者之論。以爲公歿而...,"\n오호라, 이 책은 우리 향산공(響山公)의 유문(遺文)이다. 공의 큰 절의가 이미...",如先生者更於何叫起來也,"[如, 先, 生, 者, 更, 於, 何, 叫, 起, 來, 也]"
3796190,ITKC_MO_1252A_0290_000_0030,嗚呼。此吾響山公遺文也。公大節旣昭然如日星之在天。其不朽何待乎文哉。獨後死者之論。以爲公歿而...,"\n오호라, 이 책은 우리 향산공(響山公)의 유문(遺文)이다. 공의 큰 절의가 이미...",只得拊遺卷爲之太息而已,"[只, 得, 拊, 遺, 卷, 爲, 之, 太, 息, 而, 已]"


In [ ]:
# 전체 데이터에서 진행하기
for i in tqdm(df_sent.index):
  words(df_sent['token'][i], '我國') # 문자 단위로 토크나이징 된 리스트들이 들어있는 열과 합치고 싶은 단어를 넣기
  #print(result)

df_sent.head()

In [10]:
# 단어 
word = '我國'
inword = [i for i in tqdm(df_sent.index) if word in df_sent['token'][i]]
print(inword)

100%|██████████| 3796192/3796192 [00:25<00:00, 147514.76it/s]

[5204, 13700, 22669, 32373, 34945, 38525, 38818, 38962, 39050, 39278, 42186, 42258, 42276, 42424, 42425, 42532, 42544, 42560, 42577, 42599, 42712, 42722, 42824, 42826, 42951, 42957, 42978, 42985, 42996, 43007, 43024, 43073, 43162, 43208, 43219, 43248, 43267, 43364, 43386, 43420, 47504, 48721, 48984, 52619, 52629, 54682, 56295, 57124, 57552, 60086, 60844, 60872, 61291, 66972, 75286, 75385, 88303, 100376, 111735, 114114, 136857, 139116, 139411, 139672, 142980, 143463, 151306, 153992, 169833, 171199, 184044, 190636, 192091, 192176, 192184, 195678, 195783, 196001, 196021, 196025, 196411, 196822, 197323, 197426, 197435, 197535, 200098, 200437, 202065, 203679, 204753, 204826, 204835, 205352, 205383, 205427, 205626, 205680, 206159, 206297, 206828, 207627, 208827, 211155, 217044, 224077, 224331, 226037, 226129, 226206, 226682, 232338, 233163, 239697, 241071, 241430, 241716, 241926, 242098, 242823, 243410, 244182, 244442, 245686, 277488, 277661, 279234, 280732, 282236, 282323, 282333, 282611, 2

In [11]:
#단어 포함된 데이터 프레임 확인
df_sent.loc[inword]

,ID,원문,번역문,sentence,token
5204,ITKC_MO_0001A_0120_010_0010,廣明二年七月八日。諸道都統檢校太尉某。告黃巢。夫守正修常曰道。臨危制變曰權。智者成之於順時。...,\n광명(廣明) 2년(881) 7월 8일에 제도도통 검교태위(諸道都統檢校太尉) 모(...,比者我國家德深含垢,"[比, 者, 我國, 家, 德, 深, 含, 垢]"
13700,ITKC_MO_0002A_0030_010_0180,靑鶴洞碑。鄭弘溟銘曰。若高麗，百濟，新羅。國雖一域。粤有蓬萊，瀛洲，方丈。山則三神。積氣扶桑...,"\n고려와 백제와 신라로 말하면, 나라는 비록 한 지역에 속한다고 하겠지만 봉래(蓬...",惟我國家,"[惟, 我國, 家]"
22669,ITKC_MO_0004A_0050_010_0010,世多說東明王神異之事。雖愚夫騃婦。亦頗能說其事。僕嘗聞之。笑曰。先師仲尼。不語怪力亂神。此實...,\n세상에서 동명왕(東明王)의 신통하고 이상한 일을 많이 말한다. 비록 어리석은 남...,欲使夫天下知我國本聖人之都耳,"[欲, 使, 夫, 天, 下, 知, 我國, 本, 聖, 人, 之, 都, 耳]"
32373,ITKC_MO_0004A_0210_010_0010,兒郞偉。有屋曰廩。實惟穀藏之源。在天爲星。蓋是庫樓之象。於國莫大。其制宜宏。我國家宅萬世之都...,"\n아랑위(兒郞偉), 집에 있어서는 곳집이라 하니 실상 곡식을 저장하는 근원이요, ...",我國家宅萬世之都,"[我國, 家, 宅, 萬, 世, 之, 都]"
34945,ITKC_MO_0004A_0230_020_0110,吳德全爲詩。遒邁勁俊。其詩之膾炙人口者。不爲不多。然未見能押強韻。儼若天成者。及於北山欲題戟...,\n오덕전은 시를 힘차고 준수하게 지었다. 그 시가 사람들의 입에 오르내린 것이 퍽...,我國人茫然無以對,"[我國, 人, 茫, 然, 無, 以, 對]"
...,...,...,...,...,...
3787514,ITKC_MO_1252A_0240_010_0150,宣城搴芝山南遠岱之北。有所謂招魂葬者。卽贈軍器寺僉正琴公海龍之墓也。公奉化人。高麗平章事儀之...,\n선성(宣城 예안(禮安)의 고호)의 건지산(搴芝山) 남쪽 원대(遠岱)의 북쪽에 초...,我國外亂,"[我國, 外, 亂]"
3791962,ITKC_MO_1252A_0270_000_0010,公諱晩燾字觀必號響山。又曰直齋。姓李氏眞城人。麗末以縣吏中司馬諱碩其始祖。六傳而至文純公諱滉...,"\n공은, 휘는 만도(晩燾), 자는 관필(觀必), 호는 향산(響山) 또는 직재(直齋...",而竊伏念我國與日本,"[而, 竊, 伏, 念, 我國, 與, 日, 本]"
3793265,ITKC_MO_1252A_0270_000_0030,惟我國家隆盛。退陶夫子以道學啓一治文明之運。國家將終。響山先生以節義扶萬世綱常之重。盖道學節...,\n우리나라가 융성하였을 때 퇴도(退陶) 선생이 도학으로써 일치문명(一治文明)의 운...,惟我國家隆盛,"[惟, 我國, 家, 隆, 盛]"
3794536,ITKC_MO_1252A_0280_000_0010,府君諱晩燾字觀必。號響山。又曰直齋。姓李氏。始祖諱碩。麗末以眞寶縣吏中司馬。生諱子脩。登第討...,"\n부군은 휘는 만도(晩燾), 자는 관필(觀必), 호는 향산(響山) 혹은 직제(直齋...",而竊伏念我國與日本壬辰之後,"[而, 竊, 伏, 念, 我國, 與, 日, 本, 壬, 辰, 之, 後]"


#### 단어 여러개 한번에 합치기



In [140]:
# def word_lists (lists, word_list):
#   global result
#   result =[]
#   for word in word_list :
#     char1 , char2 = list(word)
#     for index, value in enumerate(lists):
#       try: 
#         if (value == char1) & (lists[index+1]==char2):
#           #test[index] = '南地'
#           #del test[index+1]
#           result.append(word)
#           print(word)
#         elif (value ==char2)&(lists[index-1]==char1) :
#           continue
#         else :
#           result.append(value)
#       except IndexError :
#         result.append(value)
#         continue

In [12]:
def word_lists (lists, word_list):
  global result
  
  for word in word_list :
    char1 , char2 = list(word)
    for index, value in enumerate(lists):
      try: 
        if (value == char1) & (lists[index+1]==char2):
          lists[index] = word
          #print(word)
        elif (value ==char2)&(lists[index-1]==word) :
          del lists[index]
        else :
          continue
      except IndexError :
        result =lists

In [ ]:
word_list = ['我國', '天下']  #단어 단위로 토크나이징할 단어 목록 작성
for i in tqdm(df_sent.index):
  word_lists(df_sent['token'][i], word_list)

df_sent.head()

In [ ]:
#확인 
word1 = '我國'
word2 ='天下'
inword = [i for i in tqdm(df_sent.index) if (word1 in df_sent['token'][i])&(word2 in df_sent['token'][i])]
print(inword)

In [16]:
df_sent['token'][22669]

['欲', '使', '夫', '天下', '知', '我國', '本', '聖', '人', '之', '都', '耳']

In [17]:
#실제로 잘 나옴
df_sent.loc[inword]

,ID,원문,번역문,sentence,token
22669,ITKC_MO_0004A_0050_010_0010,世多說東明王神異之事。雖愚夫騃婦。亦頗能說其事。僕嘗聞之。笑曰。先師仲尼。不語怪力亂神。此實...,\n세상에서 동명왕(東明王)의 신통하고 이상한 일을 많이 말한다. 비록 어리석은 남...,欲使夫天下知我國本聖人之都耳,"[欲, 使, 夫, 天下, 知, 我國, 本, 聖, 人, 之, 都, 耳]"
1200513,ITKC_MO_0299A_0250_010_0110,臣等昨日竭誠陳啓。恭竢兪音。而竟下勿煩之敎。相顧色沮。竊忖殿下拒之之深也。臣等非欲強爲煩籲。...,\n신들은 어제 온 정성을 다해 진달하고서 윤허한다는 명이 내려지기를 공손하게 기다...,我國素以禮義見稱於天下,"[我國, 素, 以, 禮, 義, 見, 稱, 於, 天下, 下]"
1421620,ITKC_MO_0329A_0060_010_0110,\n伏以君君臣臣父父子子。天地之常經也。利則就之。害則避之。人物之私心也。君之所以仁。臣之所...,\n삼가 아룁니다. 임금은 임금답고 신하는 신하답고 아비는 아비답고 자식은 자식다운...,則天下其謂我國何,"[則, 天下, 其, 謂, 我國, 何]"
1537458,ITKC_MO_0333A_0380_010_0360,我國東界一帶海水。無潮汐。故有東海無潮汐之說。而古之人未嘗道也。余嘗思其理而不可得。竊意非東...,"\n우리나라의 동해안 일대에는 밀물과 썰물 현상이 없기 때문에, 동해(東海)에는 으...",我國處天下之東北隅,"[我國, 處, 天下, 之, 東, 北, 隅]"
1964926,ITKC_MO_0392A_0070_010_0010,鼎重再拜言。近者得聞閭巷喧傳。北人發▦差自燕城。今十六上馬。廿六到我境。問其故。乃請兵事也。...,\n정중(鼎重)은 재배하고 말씀드립니다. 근래 여항(閭巷)에서 떠들썩하게 전해지는 ...,我國之負罪於天下久矣,"[我國, 之, 負, 罪, 於, 天下, 久, 矣]"
2252863,ITKC_MO_0402A_0070_060_0040,海之有潮汐。先儒說非一。谿谷斷以地之喘息之說爲是曰。地勢北爲背而南爲腹。腹有喘息而背無喘息。...,"\n바다에는 밀물과 썰물이 있는데, 선유(先儒)들의 주장은 한 가지가 아니다. 계곡...",我國處天下之東北隅,"[我國, 處, 天下, 之, 東, 北, 隅]"


### 원문 Word2vec

In [18]:
# vector : 차원, window : 앞뒤 단어 보는 크기, min_count : 분석에 포함할 최소 단어 빈도수, sg : 0이면 cbow, 1이면 skip-gram
v300_w5_min5000_sg = Word2Vec(sentences=df_sent['token'].to_list(), vector_size=300, window=5, min_count=300,sg=1,seed=2021, workers=1)
v300_w5_min5000_sg.save('ch_v300_w5_min300_sg.model') # 모델 저장. 모델 로드할때는 model= Word2Vec.load('v200_w3_min300_sg.model')

In [19]:
v300_w5_min300_sg = Word2Vec.load('ch_v300_w5_min300_sg.model')

In [20]:
# 해당 단어와 문맥이 비슷한 유사어 추출
v300_w5_min300_sg.wv.most_similar('我國', topn=20)  # get other similar words

[('邦', 0.4506572186946869),
 ('國', 0.4392912983894348),
 ('倭', 0.42779916524887085),
 ('虜', 0.3696169853210449),
 ('奴', 0.32530370354652405),
 ('彼', 0.3199054002761841),
 ('我', 0.31551089882850647),
 ('敝', 0.2898019850254059),
 ('家', 0.2890818417072296),
 ('寇', 0.28790396451950073),
 ('據', 0.28744396567344666),
 ('搶', 0.28346264362335205),
 ('制', 0.2829420566558838),
 ('疆', 0.2820996940135956),
 ('敵', 0.280277281999588),
 ('民', 0.2778319716453552),
 ('遼', 0.2759800851345062),
 ('僻', 0.2716580033302307),
 ('賣', 0.2716001570224762),
 ('朝', 0.2712170481681824)]

In [21]:

v300_w5_min300_sg.wv.most_similar('天下', topn=20)  # get other similar words

[('世', 0.3634366989135742),
 ('國', 0.33440452814102173),
 ('民', 0.30827951431274414),
 ('人', 0.29410824179649353),
 ('體', 0.2931108772754669),
 ('足', 0.2910124361515045),
 ('者', 0.2894386053085327),
 ('爲', 0.2753329277038574),
 ('也', 0.2680206894874573),
 ('治', 0.26540684700012207),
 ('道', 0.2652415931224823),
 ('我國', 0.2597745954990387),
 ('斁', 0.259432852268219),
 ('界', 0.2568449378013611),
 ('理', 0.25576356053352356),
 ('萬', 0.25356659293174744),
 ('非', 0.2526855170726776),
 ('綜', 0.24777764081954956),
 ('好', 0.24699200689792633),
 ('法', 0.24577412009239197)]

In [22]:
# 해당 단어와 문맥이 비슷한 유사어 추출
v300_w5_min300_sg.wv.most_similar('我', topn=20)

[('吾', 0.49789440631866455),
 ('余', 0.43956294655799866),
 ('予', 0.3639432489871979),
 ('我國', 0.3155108690261841),
 ('皇', 0.299473375082016),
 ('朕', 0.29194727540016174),
 ('敝', 0.28843459486961365),
 ('人', 0.28516510128974915),
 ('王', 0.2837861180305481),
 ('瀷', 0.2747870981693268),
 ('眷', 0.2690417170524597),
 ('惟', 0.26610109210014343),
 ('寡', 0.2620546817779541),
 ('爾', 0.2617529034614563),
 ('賴', 0.2613736689090729),
 ('拯', 0.2588890790939331),
 ('哲', 0.25825026631355286),
 ('祖', 0.25808802247047424),
 ('遐', 0.2573687434196472),
 ('饗', 0.2568538784980774)]

In [23]:
v300_w5_min300_sg.wv.most_similar('吾', topn=20)

[('我', 0.49789443612098694),
 ('余', 0.47470101714134216),
 ('渠', 0.44034630060195923),
 ('汝', 0.4139038622379303),
 ('兄', 0.39445704221725464),
 ('弟', 0.359849214553833),
 ('瀷', 0.34908387064933777),
 ('此', 0.3409530222415924),
 ('儕', 0.32660409808158875),
 ('予', 0.3255632519721985),
 ('謂', 0.3214833438396454),
 ('其', 0.3208000063896179),
 ('父', 0.31493258476257324),
 ('亦', 0.31418147683143616),
 ('君', 0.3124141991138458),
 ('爲', 0.30843496322631836),
 ('姪', 0.30670425295829773),
 ('則', 0.3047868013381958),
 ('滉', 0.3034503757953644),
 ('人', 0.30026334524154663)]

In [24]:
v300_w5_min300_sg.wv.most_similar('東', topn=20)

[('西', 0.5906899571418762),
 ('南', 0.538338840007782),
 ('北', 0.44825369119644165),
 ('瀛', 0.3285297751426697),
 ('湖', 0.3262564241886139),
 ('球', 0.3061167001724243),
 ('江', 0.30467313528060913),
 ('州', 0.30424532294273376),
 ('迤', 0.2970682680606842),
 ('灣', 0.2960863709449768),
 ('海', 0.29495862126350403),
 ('山', 0.2944329082965851),
 ('邦', 0.2884521186351776),
 ('坡', 0.28770142793655396),
 ('伽', 0.2876879870891571),
 ('萊', 0.28730568289756775),
 ('河', 0.28454986214637756),
 ('濱', 0.2813985049724579),
 ('隅', 0.28070807456970215),
 ('崑', 0.27787521481513977)]

In [27]:
target_list = ['我', '吾', '東', '國', '我國', '天下']

In [28]:
res = pd.DataFrame()
for target in target_list:
    res = pd.concat([res,pd.DataFrame(v300_w5_min300_sg.wv.most_similar(target, topn=20),columns=[target,'similarity'])],axis=1)  # get other similar words
res

,我,similarity,吾,similarity,東,similarity,國,similarity,我國,similarity,天下,similarity
0,吾,0.497894,我,0.497894,西,0.590690,邦,0.452586,邦,0.450657,世,0.363437
1,余,0.439563,余,0.474701,南,0.538339,我國,0.439291,國,0.439291,國,0.334405
2,予,0.363943,渠,0.440346,北,0.448254,天下,0.334405,倭,0.427799,民,0.308280
3,我國,0.315511,汝,0.413904,瀛,0.328530,廷,0.322181,虜,0.369617,人,0.294108
4,皇,0.299473,兄,0.394457,湖,0.326256,億,0.294401,奴,0.325304,體,0.293111
5,朕,0.291947,弟,0.359849,球,0.306117,域,0.293024,彼,0.319905,足,0.291012
6,敝,0.288435,瀷,0.349084,江,0.304673,家,0.277146,我,0.315511,者,0.289439
7,人,0.285165,此,0.340953,州,0.304245,朝,0.276455,敝,0.289802,爲,0.275333
8,王,0.283786,儕,0.326604,迤,0.297068,社,0.275589,家,0.289082,也,0.268021
9,瀷,0.274787,予,0.325563,灣,0.296086,麗,0.269859,寇,0.287904,治,0.265407


In [29]:
res.to_excel('한국고전종합DB_원문_단어 합치기_word2vec.xlsx')